## Предобработка данных

In [2]:
import pandas as pd
import numpy as np

In [3]:
groups = pd.read_table('groups.txt')
groups.head()

,doc_id,variant,group_id,chain_id,link,shift,length,content,tk_shifts,attributes,head,hd_shifts
0,1,1,407840,1070,0,9,5,своих,9,ref:def|str:refl|type:coref,NaN,NaN
1,1,1,407839,1070,407840,47,1,я,47,ref:def|str:pron|type:coref,NaN,NaN
2,1,1,407842,1069,0,69,13,одинокую дачу,"69,78",ref:def|str:noun|type:coref,дачу,78
3,1,1,407841,1069,407842,118,9,этой даче,"118,123",ref:def|str:noun|type:coref,даче,123
4,1,1,407843,1069,407841,166,3,она,166,ref:def|str:pron|type:coref,NaN,NaN


In [4]:
new_groups = groups[groups.chain_id > 0].sort_values(by='chain_id') #отсортировали данные по chain_id

In [5]:
new_groups.loc[:, 'first'] = 0

In [6]:
group_ids = new_groups.groupby("chain_id").aggregate(np.min)['shift'].get_values() # группируем по chain_id и выбираем минимум по shift

for i in range(len(group_ids)):
    new_groups.loc[new_groups[(new_groups['chain_id'] == i+1)&(new_groups['shift'] == group_ids[i])].index, 'first'] = 1

In [7]:
new_groups.head()

,doc_id,variant,group_id,chain_id,link,shift,length,content,tk_shifts,attributes,head,hd_shifts,first
5903,86,1,413740,1,413739,3043,4,свои,3043,ref:def|str:refl|type:coref,NaN,NaN,0
5899,86,1,413736,1,0,2835,15,многие дарители,"2835,2842",ref:def|str:noun|type:coref,дарители,2842,1
5904,86,1,413741,1,413740,3058,3,Они,3058,ref:def|str:pron|type:coref,NaN,NaN,0
5900,86,1,413737,1,413736,2861,4,свои,2861,ref:def|str:refl|type:coref,NaN,NaN,0
5901,86,1,413739,1,413737,2922,10,Эти страны,"2922,2926",ref:def|str:noun|type:coref,страны,2926,0


### Получаем списки прилагательных

In [8]:
A = new_groups[(new_groups['first'] == 1)&(new_groups['head'] != None)]['content'].to_string().split()
A = [x.lower() for x in A if x.isalpha()]

In [9]:
from pymorphy2 import MorphAnalyzer

In [10]:
morph = MorphAnalyzer()

In [11]:
import collections

In [12]:
c = collections.Counter()

In [13]:
for word in A:
    p = morph.parse(word)[0]
    if p.tag.POS == 'ADJF':
        c[p.normal_form] += 1

In [14]:
c.most_common(30)

[('свой', 69),
 ('этот', 46),
 ('один', 45),
 ('тот', 42),
 ('весь', 37),
 ('наш', 36),
 ('новый', 19),
 ('такой', 18),
 ('американский', 17),
 ('мой', 16),
 ('самый', 16),
 ('большой', 15),
 ('европейский', 15),
 ('ядерный', 15),
 ('главный', 14),
 ('российский', 13),
 ('некоторый', 12),
 ('русский', 12),
 ('международный', 11),
 ('другой', 11),
 ('каждый', 10),
 ('народный', 10),
 ('южный', 9),
 ('хороший', 9),
 ('небольшой', 9),
 ('западный', 8),
 ('футбольный', 8),
 ('оппозиционный', 8),
 ('немецкий', 8),
 ('какой', 8)]

In [15]:
f = open('first_adjectives.txt', 'w', encoding = 'utf-8')
for item in list(c):
    f.write("%s\n" % item)
f.close()

In [16]:
B = new_groups[(new_groups['first'] == 0)&(new_groups['head'] != None)]['content'].to_string().split()
B = [x.lower() for x in B if x.isalpha()]

In [17]:
d = collections.Counter()

In [18]:
for word in B:
    p = morph.parse(word)[0]
    if p.tag.POS == 'ADJF':
        d[p.normal_form] += 1

In [19]:
d.most_common(30)

[('который', 737),
 ('свой', 543),
 ('этот', 220),
 ('мой', 126),
 ('наш', 112),
 ('тот', 66),
 ('сам', 56),
 ('её', 54),
 ('ядерный', 43),
 ('самый', 42),
 ('новый', 37),
 ('фри', 35),
 ('ваш', 32),
 ('весь', 28),
 ('один', 27),
 ('такой', 24),
 ('сундучанский', 21),
 ('большой', 21),
 ('российский', 20),
 ('южный', 15),
 ('бывший', 13),
 ('старый', 11),
 ('обыкновенный', 11),
 ('летний', 10),
 ('английский', 10),
 ('американский', 9),
 ('высокий', 9),
 ('народный', 9),
 ('общий', 9),
 ('главный', 9)]

In [20]:
c1 = c - d

In [21]:
c1.most_common(30)

[('один', 18),
 ('некоторый', 12),
 ('каждый', 9),
 ('весь', 9),
 ('американский', 8),
 ('европейский', 7),
 ('другой', 7),
 ('немецкий', 7),
 ('собственный', 6),
 ('местный', 6),
 ('оппозиционный', 6),
 ('русский', 6),
 ('мелкий', 6),
 ('общественный', 6),
 ('радикальный', 5),
 ('западный', 5),
 ('национальный', 5),
 ('хороший', 5),
 ('небольшой', 5),
 ('главный', 5),
 ('какой', 5),
 ('огромный', 5),
 ('глобальный', 5),
 ('окружающий', 5),
 ('многий', 5),
 ('радиоактивный', 5),
 ('московский', 5),
 ('наивный', 5),
 ('социальный', 4),
 ('широкий', 4)]

In [22]:
common_first_adjectives = []
for i in range (61):
    common_first_adjectives.append(c1.most_common(61)[i][0])

In [23]:
f = open('not_first_adjectives.txt', 'w', encoding = 'utf-8')
for item in list(d):
    f.write("%s\n" % item)
f.close()

### Подготовка к baseline

In [24]:
documents = pd.read_table('Documents.txt')

#### Признак "совпадение именной группы"

In [25]:
new_groups['content coincidence'] = 0

In [26]:
for i in range(181): # идём по документам
    f = open('rucoref_texts/'+ documents['path'][i], 'r', encoding = 'utf-8' )
    text = f.read()
    f.close()
    indexes = new_groups[new_groups['doc_id'] == documents['doc_id'][i]]['content'].index 
    for index in indexes: # идём по content'ам
        if text.find(new_groups['content'][index]) == new_groups['shift'][index]: #находим первое упоминание content в тексте и сверяем с настоящим местом content (shift) 
            new_groups.loc[index, 'content coincidence'] = 1 # если первое упоминание совпало с шифтом, то признак 1

#### Признак "совпадение вершины"

In [27]:
new_groups['head coincidence'] = 0

In [28]:
for i in range(181):
    f = open('rucoref_texts/'+ documents['path'][i], 'r', encoding = 'utf-8')
    text = f.read()
    f.close()
    indexes = new_groups[new_groups['doc_id'] == documents['doc_id'][i]]['head'].index
    for index in indexes:
        if str(text.find(str(new_groups['head'][index]))) == new_groups['hd_shifts'][index]:
            new_groups.loc[index, 'head coincidence'] = 1

#### Признак "является ли местоимением"

In [29]:
new_groups['is_pron'] = 1

In [30]:
indexes = new_groups['content'].index 
for index in indexes: # идём по contentам
    p = morph.parse(new_groups['content'][index])[0]
    if p.tag.POS != 'NOUN': #если ИГ -  не существительное,  
        new_groups.loc[index, 'is_pron'] = 0 # то признак = 0 (тогда вероятно, что это НЕ первое упоминание)

#### Признак "количество прилагательных"

In [31]:
new_groups['number_of_adj'] = 0

In [32]:
indexes = new_groups['content'].index 
for index in indexes: # идём по contentам
    words = new_groups['content'][index].split()
    num_adj = 0 #счетчик прилагательных
    for word in words:
        p = morph.parse(word)[0]
        if p.tag.POS == 'ADJF': #если ИГ -  прилагательное,
            num_adj += 1 #прибавляем 1
    if num_adj >= 1: #если прилагательных больше одного
        new_groups.loc[index, 'number_of_adj'] = 1 # то признак = 1

### Baseline

In [33]:
X = new_groups[['content coincidence','head coincidence', 'is_pron', 'number_of_adj']]
Y = new_groups['first']

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.3, random_state=12345)

#### Логистическая Регресссия

In [35]:
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression()

In [36]:
lr_clf.fit(X_train, y_train)

C:\Users\mikat\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [37]:
y_pred = lr_clf.predict(X_test)

In [38]:
from sklearn.metrics import classification_report
target_names = ['class 0', 'class 1']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.80      0.95      0.87      3828
     class 1       0.58      0.21      0.31      1122

   micro avg       0.79      0.79      0.79      4950
   macro avg       0.69      0.58      0.59      4950
weighted avg       0.75      0.79      0.75      4950



#### SVC

In [39]:
from sklearn.svm import SVC

lin_svm = SVC(kernel='linear', C=10)

In [40]:
lin_svm.fit(X_train, y_train)

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [41]:
y_pred1 = lin_svm.predict(X_test)

In [42]:
print(classification_report(y_test, y_pred1, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.82      0.94      0.88      3828
     class 1       0.59      0.28      0.38      1122

   micro avg       0.79      0.79      0.79      4950
   macro avg       0.71      0.61      0.63      4950
weighted avg       0.77      0.79      0.76      4950



#### KNN

In [44]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)

In [45]:
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [46]:
y_pred3 = knn.predict(X_test)

In [47]:
print(classification_report(y_test, y_pred3, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.82      0.93      0.87      3828
     class 1       0.55      0.31      0.40      1122

   micro avg       0.79      0.79      0.79      4950
   macro avg       0.69      0.62      0.63      4950
weighted avg       0.76      0.79      0.76      4950



## Эксперименты

### Длина

In [48]:
new_groups['len'] = 0

In [49]:
new_groups.length.mean()

10.011031640198812

In [50]:
new_groups.loc[new_groups.length >= 10, 'len'] = 1 #4911

In [51]:
X1 = new_groups[['content coincidence','head coincidence', 'is_pron', 'number_of_adj', 'len']]

In [52]:
X_train, X_test, y_train, y_test = train_test_split(
    X1, Y, test_size=0.3, random_state=12345)

#### KNN

In [53]:
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [54]:
y_pred = knn.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.84      0.88      0.86      3828
     class 1       0.52      0.45      0.48      1122

   micro avg       0.78      0.78      0.78      4950
   macro avg       0.68      0.66      0.67      4950
weighted avg       0.77      0.78      0.77      4950



#### Логистическая регрессия

In [55]:
lr_clf.fit(X_train, y_train)

C:\Users\mikat\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [57]:
y_pred = lr_clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.83      0.90      0.86      3828
     class 1       0.52      0.37      0.43      1122

   micro avg       0.78      0.78      0.78      4950
   macro avg       0.68      0.64      0.65      4950
weighted avg       0.76      0.78      0.77      4950



#### SVC

In [58]:
lin_svm.fit(X_train, y_train)

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [59]:
y_pred = lin_svm.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.82      0.94      0.88      3828
     class 1       0.59      0.28      0.38      1122

   micro avg       0.79      0.79      0.79      4950
   macro avg       0.71      0.61      0.63      4950
weighted avg       0.77      0.79      0.76      4950



#### Второй эксперимент с длиной

In [65]:
new_groups['len1'] = 0

In [66]:
new_groups[new_groups['first'] == 0].length.mean()

7.8485931913570655

In [68]:
new_groups.loc[new_groups.length >= 8, 'len1'] = 1

In [69]:
X1 = new_groups[['content coincidence','head coincidence', 'is_pron', 'number_of_adj', 'len1']]

In [70]:
X_train, X_test, y_train, y_test = train_test_split(
    X1, Y, test_size=0.3, random_state=12345)

#### KNN

In [71]:
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [72]:
y_pred = knn.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.85      0.84      0.85      3828
     class 1       0.48      0.50      0.49      1122

   micro avg       0.77      0.77      0.77      4950
   macro avg       0.67      0.67      0.67      4950
weighted avg       0.77      0.77      0.77      4950



#### Логистическая регрессия

In [73]:
lr_clf.fit(X_train, y_train)

C:\Users\mikat\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [74]:
y_pred = lr_clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.80      0.96      0.87      3828
     class 1       0.58      0.21      0.31      1122

   micro avg       0.79      0.79      0.79      4950
   macro avg       0.69      0.58      0.59      4950
weighted avg       0.75      0.79      0.75      4950



#### SVC

In [75]:
lin_svm.fit(X_train, y_train)

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [79]:
y_pred = lin_svm.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.82      0.94      0.88      3828
     class 1       0.59      0.28      0.38      1122

   micro avg       0.79      0.79      0.79      4950
   macro avg       0.71      0.61      0.63      4950
weighted avg       0.77      0.79      0.76      4950



Выводы по экспериментам с длиной:
    
    Для knn эффективнее использовать среднее значение повторных ИГ (видимо, из-за того, что в результате к первому классу будет относиться больше ИГ)
    
    Для логистической регрессии эффективнее использовать среднее значение по всем ИГ

### Прилагательные

In [80]:
new_groups['is_adj'] = 0

In [81]:
indexes = new_groups['content'].index 
for index in indexes: # идём по contentам
    words = new_groups['content'][index].split()
    for word in words:
        p = morph.parse(word)[0]
        if p.tag.POS == 'ADJF': #если ИГ -  прилагательное,
            if p.normal_form in common_first_adjectives:
                new_groups.loc[index, 'is_adj'] = 1 # то признак = 1

In [82]:
X2 = new_groups[['content coincidence','head coincidence', 'is_pron', 'len1', 'is_adj']]

In [83]:
X_train, X_test, y_train, y_test = train_test_split(
    X2, Y, test_size=0.3, random_state=12345)

#### KNN

In [84]:
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [85]:
y_pred = knn.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.84      0.83      0.84      3828
     class 1       0.45      0.47      0.46      1122

   micro avg       0.75      0.75      0.75      4950
   macro avg       0.64      0.65      0.65      4950
weighted avg       0.75      0.75      0.75      4950



#### Логистическая регрессия

In [86]:
X2 = new_groups[['content coincidence','head coincidence', 'is_pron', 'len', 'is_adj']]

In [87]:
X_train, X_test, y_train, y_test = train_test_split(
    X2, Y, test_size=0.3, random_state=12345)

In [88]:
lr_clf.fit(X_train, y_train)

C:\Users\mikat\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [89]:
y_pred1 = lr_clf.predict(X_test)
print(classification_report(y_test, y_pred1, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.82      0.95      0.88      3828
     class 1       0.61      0.26      0.37      1122

   micro avg       0.80      0.80      0.80      4950
   macro avg       0.71      0.61      0.62      4950
weighted avg       0.77      0.80      0.76      4950



#### SVC

In [90]:
lin_svm.fit(X_train, y_train)

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [91]:
y_pred2 = lin_svm.predict(X_test)
print(classification_report(y_test, y_pred2, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.83      0.94      0.88      3828
     class 1       0.61      0.33      0.43      1122

   micro avg       0.80      0.80      0.80      4950
   macro avg       0.72      0.64      0.66      4950
weighted avg       0.78      0.80      0.78      4950

